# Analyze colony images from the frames of the videos
# Cannot create a df of all the timepoints, the file becomes too large!

In [ ]:
%matplotlib notebook

import numpy as np
import pandas as pd
import cv2
import sys

import colony_iomethods as cm
import xml.etree.ElementTree as ET # for XML properties

from matplotlib import pyplot as plt

## Flatten the 2d numpy array and obtain the x,y indices from the image
### Numpy arrays are flattened in C' style by default. C style is column-first.

In [ ]:
def img_flatten(img, extent):
    # img needs to be 2d numpy array
    (height, width) = img.shape
    
    # Flattened image returns 
    img_flat = img.flatten()
    
    # Column-first (xix-first)
    xix = np.array(list(range(extent[2],extent[3]))*width, dtype=np.uint16)
    yix = np.array([[d]*height for d in range(extent[0],extent[1])], dtype=np.uint16).flatten()
    return (xix, yix, img_flat)

def imgflat_to_2d(imgflat_roi, xixflat_roi, yixflat_roi):
    xunique = np.unique(xixflat_roi)
    yunique = np.unique(yixflat_roi)
    img2d = np.zeros((len(yunique),len(xunique)))
    for ix in range(len(xunique)):
        for jx in range(len(yunique)):
            img2d[jx, ix] = imgflat_roi[jx + ix*len(yunique)]
    return (xunique, yunique, img2d)

## Save all the frames of an .avi video to a dataframe
* Columns include: [xix, yix, zix, tix, pixel->uint8]
* Indices are extracted from the filename for tix, frame count and xml data for xix, yix, and zix
* the values, xval, yval, zval, tval are the values of the given pixel in units of xunit, yunit, zunit, tunit, read from the xml properties file
* Will append the pixel average values from various blurring options

### File paths

In [ ]:
# Filename
fname = "3dTimeScan_17h_init_long"
tstrar = ["t%02d" % (tix) for tix in range(48)]

# Working folder
folder = "C:/Users/Tolga/Dropbox/Raw Data/Colony Images - Confocal/02112020/" + fname + "/"

#Video folder and path
vidfolder = "Videos/"

# XML properties file
xml_name = "Metadata/EQ59_Single_Colony_TilesScan.lif_" + fname + "_Properties.xml"
xml_path = folder + xml_name

### Important!! 
#### Each frame is scaled from a tilescan of 1024x1024 images, to a final 512x512 in the video.
* Collect the tilescan positions and obtain the scalex and scaley

* Scale xvoxel and yvoxel accordingly

### Read XML properties

In [ ]:
tree = ET.parse(xml_path)    # xml tree of the current stage position
root = tree.getroot()           # root of the xml tree

image_xml = root[0]

# Tilescan properties
tilescan_desc = cm.collectTileScan(image_xml)
# Number of tiles in x- and y- directions
tile_xcnt = tilescan_desc["tile_xcnt"]
tile_ycnt = tilescan_desc["tile_ycnt"]
# Scale factors
scalex = 1.0/tile_xcnt/2
scaley = 1.0/tile_ycnt/2

dim_desc = cm.collectImageDim(image_xml)
xsz = dim_desc["xsz"]
ysz = dim_desc["ysz"]
zsz = dim_desc["zsz"]
tsz = dim_desc["tsz"]
xvoxel = dim_desc["xvoxel"]/scalex
yvoxel = dim_desc["yvoxel"]/scaley
zvoxel = dim_desc["zvoxel"]
xunit = dim_desc["xunit"]
yunit = dim_desc["yunit"]
zunit = dim_desc["zunit"]

### Read and save each frame of the video

In [ ]:
# ROI in um
xsmall = 443
xlarge = 3200
ysmall = 450
ylarge = 3220

# Accepted ROI is in range xsmall, xlarge, ysmall, ylarge
# Calculate the corresponding indices: xixsmall, xixlarge, yixsmall, yixlarge
# xvoxel*xixsmall = xsmall
xixsmall = np.uint16(xsmall/xvoxel)
xixlarge = np.uint16(xlarge/xvoxel)
yixsmall = np.uint16(ysmall/yvoxel)
yixlarge = np.uint16(ylarge/yvoxel)
extent = [xixsmall, xixlarge, yixsmall, yixlarge]

# Dictionary for DataFrame creating, define the columns
df_dict = {"xix": np.empty(0, dtype = np.uint16),
           "yix": np.empty(0, dtype = np.uint16),
           "zix": np.empty(0, dtype = np.uint8),
           "tix": np.empty(0, dtype = np.uint8), 
           "pixel": np.empty(0, dtype = np.uint8)}

tvoxel = 0.5 # 30 minutes time intervals
dfall = pd.DataFrame()
for tstr in tstrar:
    # Videopath
    vidname = fname + "_" + tstr + ".avi"
    vidpath = folder + vidfolder + vidname
    
    # VideoCapture object for reading
    vidcap = cv2.VideoCapture(vidpath)
    # Frame properties
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    frame_width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))

    # Each frame shares the same x,y values at the same pixel position--> Run through each z (frame) and t
    xvals_unique = np.arange(0,frame_width*xvoxel, xvoxel)
    yvals_unique = np.arange(0,frame_height*yvoxel, yvoxel)
    
    # Time identifier
    tix = np.uint8(tstr[1:])
    # Run through each frame of the video
    for frix in range(frame_count):
        print("\rSaving time %d, frame %d" % (tix, frix), end="")
        sys.stdout.flush()

        zval = frix*zvoxel
        success,frame = vidcap.read()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        img_roi = frame_gray[yixsmall:yixlarge, xixsmall:xixlarge]   # small inclusie, large exclusive
        xixflat_roi, yixflat_roi, imgflat_roi = img_flatten(img_roi, extent)

        df_dict["xix"] = np.append(df_dict.get("xix"), xixflat_roi)
        df_dict["yix"] = np.append(df_dict.get("yix"), yixflat_roi)
        df_dict["zix"] = np.append(df_dict.get("zix"), np.array([frix]*len(xixflat_roi), dtype=np.uint8))
        df_dict["tix"] = np.append(df_dict.get("tix"), np.array([tix]*len(xixflat_roi), dtype=np.uint8))
        df_dict["pixel"] = np.append(df_dict.get("pixel"), imgflat_roi)
        
    # Save data to a dataframe
    dfall = dfall.append(pd.DataFrame(df_dict), ignore_index = True)
    # Save to a csv file
#     csvfolder = folder + "csv_data/"
#     csvpath = csvfolder + fname + "_" + tstr + ".csv"
#     df.to_csv(csvpath, index=False)

In [ ]:
csvfolder = folder + "csv_data/"
csvpath = csvfolder + fname + "_" + tstr + ".csv"
    

In [ ]:
csvfolder

In [ ]:
csvpath